#### Importing libraries

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from tqdm import tqdm

#### Load data

In [ ]:
#read in data
X_train = pd.read_csv('train_features.csv')
X_test = pd.read_csv('test_features.csv')

y_train = pd.read_csv('train_targets_scored.csv').drop(columns=['sig_id'])

In [ ]:
def encode_dummies(df):
    df['treatment'] = 1*(df.cp_type == 'trt_cp')
    df['D1'] = 1*(df.cp_dose == 'D1')
    df['cp_time'] = df.cp_time/24
    return df.drop(columns = ['sig_id', 'cp_type', 'cp_dose'])

In [ ]:
X_train = encode_dummies(X_train)
X_test = encode_dummies(X_test)

gene_list = ['g-'+str(i) for i in range(772)]
gene_exp = X_train[gene_list].values

cell_list = ['c-'+str(i) for i in range(100)]
cell_via = X_train[cell_list].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.utils import resample
from joblib import dump, load
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
def pca_features(df, var_list, n):
    '''
    df: dataframe
    var_list: variable list to be fitted
    n: number of components

    return: (fitted PCA model, principal components)
    '''
    pca = PCA(n_components=n)
    pca_features = pca.fit_transform(df[var_list])
    return (pca, pca_features)

def df_to_array(df, var_list, gene, cell):
    '''
    return: numpy array with transformed principal components
    '''
    a = df.drop(columns=var_list).values
    a = np.hstack((a, gene, cell))
    return a

def pca_transform(X_train_2, X_train_val):
    '''
    Run PCA on training data and transform both train and test data according to obtained components

    X_train_2: dataframe, training data 
    X_train_val: dataframe, test/validation data

    return: X_train, X_val, y_train, y_val as numpy array
    '''

    # X_train_2, X_train_val, y_train_2, y_train_val = train_test_split(X_train, y_train, test_size = 0.2, random_state=47)

    gene_list = ['g-'+str(i) for i in range(772)]
    cell_list = ['c-'+str(i) for i in range(100)]

    pca_gene, gene_features = pca_features(X_train_2, gene_list, 50)
    pca_cell, cell_features = pca_features(X_train_2, cell_list, 10)

    X_train_2 = df_to_array(X_train_2, gene_list+cell_list, gene_features, cell_features)
    X_train_val = df_to_array(X_train_val, gene_list+cell_list, pca_gene.transform(X_train_val[gene_list]), pca_cell.transform(X_train_val[cell_list]))

    # y_train_2 = y_train_2.values
    # y_train_val = y_train_val.values

    return X_train_2, X_train_val

def upsample(X, y, i):
    '''
    upsample positive response in the ith column of y matrix to size 1000
    '''
    # indices of positive response in the ith column
    idx1 = np.where(y[:,i]==1)[0]
    # indices of negative response in the ith column
    idx0 = np.where(y[:,i]==0)[0]
    # sample with replacement 
    up_idx = np.random.choice(idx1, replace = True, size = 1000)
    X_upsampled = np.vstack((X[idx0], X[up_idx.T]))
    y_upsampled = np.vstack(((y[idx0, i:(i+1)], y[up_idx.T, i:(i+1)])))

    return X_upsampled, y_upsampled[:,0]

#### Fitting Baseline Random Forest

In [ ]:
X_train_2, X_test_2 = pca_transform(X_train, X_test)
y_train_2 = y_train.values

rf = RandomForestClassifier()

rf.fit(X_train_2, y_train_2)

y_pred_prob = rf.predict_proba(X_test_2)

y_pred = []
for i in tqdm(range(len(y_pred_prob))):
    y_pred.append(y_pred_prob[i][:,1])

y_pred_rf = np.array(y_pred).T

#### Grid Search for RF

In [ ]:
from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 20, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 20, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
rf_Grid = GridSearchCV(estimator = rf, 
                       param_grid = random_grid,
                       cv = 3,
                       verbose = 2,
                       n_jobs = -1)


# Fit the random search model
rf_Grid.fit(X_train_2, y_train_2)


y_pred_prob_Grid = rf_Grid.predict_proba(X_test_2)


y_pred_Grid = []
for i in tqdm(range(len(y_pred_prob_Grid))):
    y_pred_Grid.append(y_pred_prob_Grid[i][:,1])

y_pred_rf_Grid = np.array(y_pred_Grid).T

#### Fitting Neural Network

In [ ]:
import torch
from torch import nn, optim
import torch.nn.functional as F 
import torch.utils.data as data

In [ ]:
X_train.values

In [ ]:
train_data = data.TensorDataset(torch.from_numpy(X_train.values), torch.from_numpy(y_train.values))
trainloader = data.DataLoader(train_data, batch_size = 256, shuffle=True)

test_data = torch.from_numpy(X_test.values)

In [ ]:
class Net(nn.Module):
  
    def __init__(self, n_features, drop_rate):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(n_features, 200)
        self.linear2 = nn.Linear(200, 180)
        self.linear3 = nn.Linear(180, 150)
        self.linear4 = nn.Linear(150, 200)
        self.linear5 = nn.Linear(200, 206)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.dropout(x, p=drop_rate)
        x = F.relu(self.linear2(x))
        x = F.dropout(x, p=drop_rate)
        x = F.relu(self.linear3(x))
        x = F.dropout(x, p=drop_rate)
        x = F.relu(self.linear4(x))
        x = F.dropout(x, p=drop_rate)
        x = self.linear5(x)

        return torch.sigmoid(x)

In [ ]:
drop_rate = 0.18
net = Net(X_train.shape[1], drop_rate)

criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0008, weight_decay=1e-5)

for epoch in range(15):

    train_loss = []

    for batch in trainloader:

        optimizer.zero_grad()

        y_pred = net(batch[0].float())
        loss = criterion(y_pred, batch[1].float())
        loss.backward()
        optimizer.step()

        train_loss.append(loss.item())
        
print("training loss:", np.mean(train_loss))

In [ ]:
# Generate predictions on test features
net.eval()
y_pred = net(test_data.float())
y_pred_nn = y_pred.detach().numpy()


In [ ]:
y_pred = (y_pred_rf+y_pred_nn)/2

In [ ]:
# Convert to submission format
col = y_train.columns
y_pred_df = pd.DataFrame(y_pred, columns = col)
sig_id = pd.read_csv('test_features.csv')[['sig_id']]
y_pred_df = sig_id.join(y_pred_df)

#write to output
y_pred_df.to_csv('submission.csv', index=False)